In [1]:
#CUSTOMER lIFETIME VALUE EXAMPLE
import pandas as pd
import datetime as dt
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [3]:
pd.set_option("display.max_columns", None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df = pd.read_excel(r"C:\Users\MONSTER\Desktop\dataset\online_retail.xlsx")

In [4]:
df = df[~df["InvoiceNo"].str.contains("C", na=False)]

In [13]:
df = df[df["Quantity"] > 0]

In [17]:
df.dropna(inplace=True)

In [19]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,397924.000,13.022,1.000,2.000,6.000,12.000,80995.000,180.420
InvoiceDate,397924,2011-07-10 23:43:36.912475648,2010-12-01 08:26:00,2011-04-07 11:12:00,2011-07-31 14:39:00,2011-10-20 14:33:00,2011-12-09 12:50:00,NaN
UnitPrice,397924.000,3.116,0.000,1.250,1.950,3.750,8142.750,22.097
CustomerID,397924.000,15294.315,12346.000,13969.000,15159.000,16795.000,18287.000,1713.170


In [23]:
df["TotalPrice"] = df["Quantity"] * df["UnitPrice"]

In [25]:
cltv_c = df.groupby("CustomerID").agg({"InvoiceNo": lambda x: x.nunique(),
                                       "Quantity": lambda x: x.sum(),
                                       "TotalPrice": lambda x: x.sum()})

In [27]:
cltv_c.columns = ["total_transaction", "total_unit", "total_price"]

In [29]:
cltv_c.describe().T

,count,mean,std,min,25%,50%,75%,max
total_transaction,4339.000,4.272,7.705,1.000,1.000,2.000,5.000,210.000
total_unit,4339.000,1194.214,5055.603,1.000,160.000,379.000,993.000,197491.000
total_price,4339.000,2053.793,8988.248,0.000,307.245,674.450,1661.640,280206.020


In [31]:
cltv_c["average_order_value"] = cltv_c["total_price"]/ cltv_c["total_transaction"]
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value
CustomerID,,,,
12346.000,1,74215,77183.600,77183.600
12347.000,7,2458,4310.000,615.714
12348.000,4,2341,1797.240,449.310
12349.000,1,631,1757.550,1757.550
12350.000,1,197,334.400,334.400


In [33]:
cltv_c["purchase_frequency"] = cltv_c["total_transaction"]/ cltv_c.shape[0]
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency
CustomerID,,,,,
12346.000,1,74215,77183.600,77183.600,0.000
12347.000,7,2458,4310.000,615.714,0.002
12348.000,4,2341,1797.240,449.310,0.001
12349.000,1,631,1757.550,1757.550,0.000
12350.000,1,197,334.400,334.400,0.000


In [35]:
repeat_rate = cltv_c[cltv_c["total_transaction"] > 1].shape[0] / cltv_c.shape[0]
churn_rate = 1 - repeat_rate

In [37]:
###sabit değer şirket tarafından belirlenir burada, 0.10 olarak belirledik
cltv_c["profit_margin"] = cltv_c["total_price"] * 0.10

In [39]:
cltv_c["customer_value"] = cltv_c["average_order_value"] * cltv_c["purchase_frequency"]

In [41]:
cltv_c["cltv"] = cltv_c["customer_value"] / churn_rate * cltv_c["profit_margin"]

In [43]:
cltv_c.sort_values(by="cltv", ascending = False).head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv
CustomerID,,,,,,,,
14646.000,74,197491,280206.020,3786.568,0.017,28020.602,64.578,5255382.439
18102.000,60,64124,259657.300,4327.622,0.014,25965.730,59.843,4512845.612
17450.000,46,69993,194550.790,4229.365,0.011,19455.079,44.838,2533467.864
16446.000,2,80997,168472.500,84236.250,0.000,16847.250,38.827,1899798.076
14911.000,201,80515,143825.060,715.548,0.046,14382.506,33.147,1384581.518


In [45]:
cltv_c

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv
CustomerID,,,,,,,,
12346.000,1,74215,77183.600,77183.600,0.000,7718.360,17.788,398748.869
12347.000,7,2458,4310.000,615.714,0.002,431.000,0.993,1243.380
12348.000,4,2341,1797.240,449.310,0.001,179.724,0.414,216.203
12349.000,1,631,1757.550,1757.550,0.000,175.755,0.405,206.759
12350.000,1,197,334.400,334.400,0.000,33.440,0.077,7.485
...,...,...,...,...,...,...,...,...
18280.000,1,45,180.600,180.600,0.000,18.060,0.042,2.183
18281.000,1,54,80.820,80.820,0.000,8.082,0.019,0.437
18282.000,2,103,178.050,89.025,0.000,17.805,0.041,2.122


In [47]:
cltv_c["segment"] = pd.qcut(cltv_c["cltv"], 4, labels = ["D","C","B","A"])

In [49]:
cltv_c.sort_values(by="cltv", ascending = False).head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv,segment
CustomerID,,,,,,,,,
14646.000,74,197491,280206.020,3786.568,0.017,28020.602,64.578,5255382.439,A
18102.000,60,64124,259657.300,4327.622,0.014,25965.730,59.843,4512845.612,A
17450.000,46,69993,194550.790,4229.365,0.011,19455.079,44.838,2533467.864,A
16446.000,2,80997,168472.500,84236.250,0.000,16847.250,38.827,1899798.076,A
14911.000,201,80515,143825.060,715.548,0.046,14382.506,33.147,1384581.518,A


In [51]:
cltv_c.groupby("segment").agg({"count", "sum", "mean"})

C:\Users\MONSTER\AppData\Local\Temp\ipykernel_14392\391531041.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cltv_c.groupby("segment").agg({"count", "sum", "mean"})


total_transaction               total_unit                    \
                    count   mean    sum      count     mean      sum   
segment                                                                
D                    1085  1.265   1373       1085  129.487   140493   
C                    1085  1.876   2035       1085  286.297   310632   
B                    1084  3.510   3805       1084  649.441   703994   
A                    1085 10.436  11323       1085 3711.131  4026577   

        total_price                      average_order_value          \
              count     mean         sum               count    mean   
segment                                                                
D              1085  178.922  194130.090                1085 154.896   
C              1085  464.366  503836.852                1085 303.783   
B              1084 1071.859 1161895.311                1084 398.158   
A              1085 6499.120 7051545.651                1085 819.334   

                   purchase_frequency             profit_margin          \
               sum              count  mean   sum         count    mean   
segment                                                                   
D       168062.431               1085 0.000 0.316          1085  17.892   
C       329604.948               1085 0.000 0.469          1085  46.437   
B       431603.305               1084 0.001 0.877          1084 107.186   
A       888977.293               1085 0.002 2.610          1085 649.912   

                   customer_value                 cltv                         
               sum          count  mean      sum count      mean          sum  
segment                                                                        
D        19413.009           1085 0.041   44.741  1085     2.497     2709.108  
C        50383.685           1085 0.107  116.118  1085    15.246    16541.709  
B       116189.531           1084 0.247  267.780  1084    82.223    89129.415  
A       705154.565           1085 1.498 1625.155  1085 22649.367 24574563.076

In [ ]:
cltv_c.to.csv("cltv_c.csv")